In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import folium
import re
import plotly.express as px
import unicodedata

In [36]:
def load_all_data(df):
    # Select AC, AD, an I columns
    indices= df[df.columns[-76:-3]]
    
    # Extract hour from column
    Col = df[df.columns[-76:-3]].columns.str.split('_',expand=True).values
    
    # Create a multi-level index to use for pivot 
    # Talk about multi index
    indices.columns = pd.MultiIndex.from_tuples([('', x[0]) if pd.isnull(x[1]) else x for x in Col])
    
    # Groupby Day of Week
    a = indices.groupby([( '', 'DoW')]).mean()
    
    # Pivot table so that extracted hour becomes a row value under HoD column for each DoW
    # Values are separated and aggregated by mean values for each of AC, AD, and I
    new = a.unstack(level=3).reset_index().dropna().rename(columns={'level_0':'index', 
                                                                'level_1':'HoD', 
                                                                ('','DoW'):'DoW',
                                                                0:'values'}).pivot_table('values',
                                                                                         ['DoW','HoD'],
                                                                                         ['index'], 
                                                                                         aggfunc='mean').reset_index()
    # Only need Days of Week Values in Heatmap therefore total can be removed
    new = new[~(new['DoW'] == 'total')]
    
    return new

In [37]:
# Enter the file path to any index file
df = pd.read_csv('../data/MM-ASP-Astral-SPUD-fyzt8738hx210315_igps19n2-ll.csv')
# switch to sample file

# CLEAN UP
df = df.rename(columns={'day_of_week':'DoW',
                        'long':'lon',
                        'audience_composition':'audComp',
                       'bell_mobile_device_match_rate':'bellMobileDeviceMatchRate'})

# Remove non-integer characters to use value for popup in Maps
df['audComp'] = df['audComp'].str.replace('%', '').astype(float)
df['index'] =df['index'].str.replace('%', '').astype(float)

# Condense Index values to average over all hours
df['AC'] = df[[s for s in df.columns if 'AC' in s]].mean(axis=1)
df['AD'] = df[[s for s in df.columns if 'AD' in s]].mean(axis=1)
df['I'] = df[[s for s in df.columns if 'I' in s]].mean(axis=1)

# Bridging Web UI into Notebooks

**Star: `ipywidgets`**

In [4]:
import ipywidgets as widgets
# From here on will use 'widgets.' to reference this library

# Introduction

We have previously looked at creating interactive visuals [07-interactive-visuals](07-interactive-visualizations.ipynb) utilizing tools/libraries such as [Plotly] and [Deck.gl]. This tutorial will take a look at the foundational component, `ipywidget`, allowing for the interactivity of these tools. The `ipywidget` library introduces python users to a new face of the python environment, where they can are given the opportunity to control and visualize data without the necessity to continuously run code. For a non-tech consumer this provides them with a level of confidence to utilize tools such as Notebook, by allowing them to see how changing inputs can be immediately reflected in different visuals.  


**In this tutorial we will examine:**
* **General Overiview of `ipywidget`**
* **Introduce different levels of `ipywidget`use cases**
* **Demonstrate the high customization capability of the tool**


## What is `ipywidgets` in a nutshell?
`ipywidget` allows us to create [interactive HTML widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) to manipulate and visualize data. Add pictures

## Use Cases:

### 0. Basic HTML Rendering

In [5]:
widgets.HTML('''
    "<i>Battlecruiser operational!</i>"
    <br />
    - A battlecruiser commander
''')

HTML(value='\n    "<i>Battlecruiser operational!</i>"\n    <br />\n    - A battlecruiser commander\n')

In [6]:
# some more nuanced HTML rendering, such as EQ product's login email template
widgets.HTML('''
  <!DOCTYPE html>
  <html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office">
  <head>
    <meta charset="utf-8"> <!-- utf-8 works for most cases -->
    <meta name="viewport" content="width=device-width"> <!-- Forcing initial-scale shouldn't be necessary -->
    <meta http-equiv="X-UA-Compatible" content="IE=edge"> <!-- Use the latest (edge) version of IE rendering engine -->
    <meta name="x-apple-disable-message-reformatting">  <!-- Disable auto-scale in iOS 10 Mail entirely -->
    <title></title> <!-- The title tag shows in email notifications, like Android 4.4. -->
    <!-- Web Font / @font-face : BEGIN -->
    <!-- NOTE: If web fonts are not required, lines 10 - 27 can be safely removed. -->
    <!-- Desktop Outlook chokes on web font references and defaults to Times New Roman, so we force a safe fallback font. -->
    <!--[if mso]>
      <style>
        * {
          font-family: sans-serif !important;
        }
      </style>
    <![endif]-->
    <!-- All other clients get the webfont reference; some will render the font and others will silently fail to the fallbacks. More on that here: http://stylecampaign.com/blog/2015/02/webfont-support-in-email/ -->
    <!--[if !mso]><!-->
    <!-- insert web font reference, eg: <link href='https://fonts.googleapis.com/css?family=Roboto:400,700' rel='stylesheet' type='text/css'> -->
    <!--<![endif]-->
    <!-- Web Font / @font-face : END -->
    <!-- CSS Reset : BEGIN -->
    <style>
      /* What it does: Remove spaces around the email design added by some email clients. */
      /* Beware: It can remove the padding / margin and add a background color to the compose a reply window. */
      html,
      body {
        margin: 0 auto !important;
        padding: 0 !important;
        height: 100% !important;
        width: 100% !important;
      }
      /* What it does: Stops email clients resizing small text. */
      * {
        -ms-text-size-adjust: 100%;
        -webkit-text-size-adjust: 100%;
      }
      /* What it does: Centers email on Android 4.4 */
      div[style*="margin: 16px 0"] {
        margin: 0 !important;
      }
      /* What it does: Stops Outlook from adding extra spacing to tables. */
      table,
      td {
        mso-table-lspace: 0pt !important;
        mso-table-rspace: 0pt !important;
      }
      /* What it does: Fixes webkit padding issue. Fix for Yahoo mail table alignment bug. Applies table-layout to the first 2 tables then removes for anything nested deeper. */
      table {
        border-spacing: 0 !important;
        border-collapse: collapse !important;
        table-layout: fixed !important;
        margin: 0 auto !important;
      }
      table table table {
        table-layout: auto;
      }
      /* What it does: Uses a better rendering method when resizing images in IE. */
      img {
        -ms-interpolation-mode:bicubic;
      }
      /* What it does: Prevents Windows 10 Mail from underlining links despite inline CSS. Styles for underlined links should be inline. */
      a {
        text-decoration: none;
      }
      /* What it does: A work-around for email clients meddling in triggered links. */
      *[x-apple-data-detectors],  /* iOS */
      .unstyle-auto-detected-links *,
      .aBn {
        border-bottom: 0 !important;
        cursor: default !important;
        color: inherit !important;
        text-decoration: none !important;
        font-size: inherit !important;
        font-family: inherit !important;
        font-weight: inherit !important;
        line-height: inherit !important;
      }
      /* What it does: Prevents Gmail from displaying a download button on large, non-linked images. */
      .a6S {
        display: none !important;
        opacity: 0.01 !important;
      }
      /* If the above doesn't work, add a .g-img class to any image in question. */
      img.g-img + div {
        display: none !important;
      }
      /* What it does: Removes right gutter in Gmail iOS app: https://github.com/TedGoas/Cerberus/issues/89  */
      /* Create one of these media queries for each additional viewport size you'd like to fix */
      /* iPhone 4, 4S, 5, 5S, 5C, and 5SE */
      @media only screen and (min-device-width: 320px) and (max-device-width: 374px) {
        .email-container {
          min-width: 320px !important;
        }
      }
      /* iPhone 6, 6S, 7, 8, and X */
      @media only screen and (min-device-width: 375px) and (max-device-width: 413px) {
        .email-container {
          min-width: 375px !important;
        }
      }
      /* iPhone 6+, 7+, and 8+ */
      @media only screen and (min-device-width: 414px) {
        .email-container {
          min-width: 414px !important;
        }
      }
    </style>
    <!-- CSS Reset : END -->
    <!-- Reset list spacing because Outlook ignores much of our inline CSS. -->
    <!--[if mso]>
    <style type="text/css">
      ul,
      ol {
        margin: 0 !important;
      }
      li {
        margin-left: 30px !important;
      }
      li.list-item-first {
        margin-top: 0 !important;
      }
      li.list-item-last {
        margin-bottom: 10px !important;
      }
    </style>
    <![endif]-->
    <!-- Progressive Enhancements : BEGIN -->
    <style>
      /* What it does: Hover styles for buttons */
      .button-td,
      .button-a {
        transition: all 100ms ease-in;
      }
      .button-td-primary:hover,
      .button-a-primary:hover {
        background: #555555 !important;
        border-color: #555555 !important;
      }
      /* Media Queries */
      @media screen and (max-width: 600px) {
        /* What it does: Adjust typography on small screens to improve readability */
        .email-container p {
          font-size: 17px !important;
        }
      }
    </style>
    <!-- Progressive Enhancements : END -->
    <!-- What it does: Makes background images in 72ppi Outlook render at correct size. -->
    <!--[if gte mso 9]>
    <xml>
      <o:OfficeDocumentSettings>
        <o:AllowPNG/>
        <o:PixelsPerInch>96</o:PixelsPerInch>
      </o:OfficeDocumentSettings>
    </xml>
    <![endif]-->
  </head>
  <!--
    The email background color (#ffffff) is defined in three places:
    1. body tag: for most email clients
    2. center tag: for Gmail and Inbox mobile apps and web versions of Gmail, GSuite, Inbox, Yahoo, AOL, Libero, Comcast, freenet, Mail.ru, Orange.fr
    3. mso conditional: For Windows 10 Mail
  -->
  <body width="100%" style="margin: 0; padding: 0 !important; mso-line-height-rule: exactly; background-color: #ffffff;">
    <center style="width: 100%; background-color: #ffffff;">
    <!--[if mso | IE]>
    <table role="presentation" border="0" cellpadding="0" cellspacing="0" width="100%" style="background-color: #ffffff;">
    <tr>
    <td>
    <![endif]-->
      <!-- Visually Hidden Preheader Text : BEGIN -->
      <div style="display: none; font-size: 1px; line-height: 1px; max-height: 0px; max-width: 0px; opacity: 0; overflow: hidden; mso-hide: all; font-family: sans-serif;">
        LOCUS (EQ Works) Login Magic Link
      </div>
      <!-- Visually Hidden Preheader Text : END -->
      <!-- Create white space after the desired preview text so email clients don’t pull other distracting text into the inbox preview. Extend as necessary. -->
      <!-- Preview Text Spacing Hack : BEGIN -->
      <div style="display: none; font-size: 1px; line-height: 1px; max-height: 0px; max-width: 0px; opacity: 0; overflow: hidden; mso-hide: all; font-family: sans-serif;">
        &zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
      </div>
      <!-- Preview Text Spacing Hack : END -->
      <!--
        Set the email width. Defined in two places:
        1. max-width for all clients except Desktop Windows Outlook, allowing the email to squish on narrow but never go wider than 600px.
        2. MSO tags for Desktop Windows Outlook enforce a 600px width.
      -->
      <div style="max-width: 600px; margin: 0 auto;" class="email-container">
        <!--[if mso]>
        <table align="center" role="presentation" cellspacing="0" cellpadding="0" border="0" width="600">
        <tr>
        <td>
        <![endif]-->
        <!-- Email Body : BEGIN -->
        <table align="center" role="presentation" cellspacing="0" cellpadding="0" border="0" width="100%" style="margin: 0 auto;">
          <!-- 1 Column Text + Button : BEGIN -->
          <tr>
            <td style="background-color: #ffffff;">
              <table role="presentation" cellspacing="0" cellpadding="0" border="0" width="100%">
                <tr>
                  <td style="padding: 10px; font-family: sans-serif; font-size: 15px; line-height: 20px; color: #555555; text-align: center;">
                    <h1 style="margin: 0 0 10px 0; font-family: sans-serif; font-size: 25px; line-height: 30px; color: #333333; font-weight: normal;">Welcome to LOCUS (EQ Works)</h1>
                  </td>
                </tr>
                <tr>
                  <td style="padding: 0 20px;">
                    <!-- Button : BEGIN -->
                    <table align="center" role="presentation" cellspacing="0" cellpadding="0" border="0" style="margin: auto;">
                      <tr>
                        <td class="button-td button-td-primary" style="border-radius: 4px; background: #6BA4F8;">
                          <a class="button-a button-a-primary" href="https://console.locus.place" style="background: #6BA4F8; border: 1px solid #DDDDDD; font-family: sans-serif; font-size: 18px; line-height: 18px; text-decoration: none; padding: 17px 20px; color: #ffffff; display: block; border-radius: 4px;">Login with the Magic Link on this device</a>
                        </td>
                      </tr>
                    </table>
                    <!-- Button : END -->
                  </td>
                </tr>
              </table>
            </td>
          </tr>
          <!-- 1 Column Text + Button : END -->
        </table>
        <!-- Email Body : END -->
        <!-- Email Footer : BEGIN -->
        <table align="center" role="presentation" cellspacing="0" cellpadding="0" border="0" width="100%" style="margin: 0 auto;">
          <tr>
            <td style="padding: 5px; font-family: sans-serif; font-size: 12px; line-height: 15px; text-align: center; color: #888888;">
              <hr>
              <p>Having an issue? <a style="color: #6BA4F8;" href="mailto:dev@eqworks.com?subject=LOCUS (EQ Works) Login issue">Contact Us</a></p>
            </td>
          </tr>
        </table>
        <!-- Email Footer : END -->
        <!--[if mso]>
        </td>
        </tr>
        </table>
        <![endif]-->
      </div>
    <!--[if mso | IE]>
    </td>
    </tr>
    </table>
    <![endif]-->
    </center>
  </body>
  </html>
''')

HTML(value='\n  <!DOCTYPE html>\n  <html lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-m…

### 1. Adding Interactivity to Variables

In [39]:
df.head()

,uid_panel,sales_market,media_product_name,board name,sales_address,lat,lon,lat_long,height,width,...,I_17,I_18,I_19,I_20,I_21,I_22,I_23,AC,AD,I
0,1786956.0,Quebec CMA,Horizontal,NaN,Pierre-Bertrand (Vanier) CL 20m AND Etienne-Du...,46.828256,-71.276132,46.828256;-71.276132,10.0,20.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000
1,1786990.0,Quebec CMA,Horizontal,NaN,Pierre-Bertrand (Vanier) CL 20m AND Etienne-Du...,46.828670,-71.277320,46.828670;-71.277320,10.0,20.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004409,0.500000,0.595052
2,1872553.0,Ottawa CMA,Horizontal,NaN,Carling Ave. (1657) (Ottawa) NS 150m W/O Churc...,45.380300,-75.748120,45.380300;-75.748120,10.0,20.0,...,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000
3,18843.0,Toronto CMA,Vertical,NaN,Bathurst St. (358) (Tor. Core) WS 45m N/O Dund...,43.652683,-79.406488,43.652683;-79.406488,16.0,12.0,...,2.698613,1.756428,2.312788,0.000000,1.493556,1.886477,2.535435,0.221981,0.058824,2.081741
4,20987.0,Toronto CMA,Vertical,NaN,Danforth Ave. (1162) (Tor.) NS 55m W/O Greenwo...,43.681241,-79.332956,43.681241;-79.332956,16.0,12.0,...,1.182405,1.168966,3.000074,2.336995,1.398671,1.834968,2.505073,0.293653,0.047619,2.595137


In [40]:
Market = widgets.Dropdown(
             options = df.sales_market.unique(),
             value = 'Toronto CMA',
             desrciptions = 'Market')
display(Market)

Dropdown(index=2, options=('Quebec CMA', 'Ottawa CMA', 'Toronto CMA', 'Montreal CMA', 'Airport Mtl', 'Trois-Ri…

### 2. Adding Interactivity to Simple Plot (Seaborn)

In [41]:
# Day of Week vs. Hour of Day  - Index Based Heatmap
def dvsh(c):
    g = sns.heatmap(load_all_data(df[df.sales_market == Market.value]).pivot_table(c,
                                        'HoD',
                                        'DoW'))
    
    g.set_title(Market.value +'\n Index: '+ c)

    return plt.show(g)
    
# Options for values included here
temp = widgets.interact(dvsh,
    c = widgets.ToggleButtons(
    options=['AC', 'AD', 'I'],
    description='Index:',
    disabled=False,
    button_style=''))

interactive(children=(ToggleButtons(description='Index:', options=('AC', 'AD', 'I'), value='AC'), Output()), _…

### 3. Enhancing the interactivity of an interactive plot (Plotly)

In [42]:
def sunburst(index):
    # Cleaning of data to represent in the graph
    # Done at this stage because the 0s and Nans are still used in mean aggregation for other visuals.
    n = df[(df[str(index)] != 0)& (df[str(index)].notna())& (df.DoW != 'total')]
    fig = px.sunburst(n, path=['sales_market', 'media_product_name', 'DoW'], values=index,
                  color=index,
                  color_continuous_scale='RdBu')
    return fig.show()
    
# Options for values included here
temp = widgets.interact(sunburst,
    index = widgets.ToggleButtons(
    options=['AC', 'AD', 'I'],
    description='Index:',
    disabled=False,
    button_style=''))

interactive(children=(ToggleButtons(description='Index:', options=('AC', 'AD', 'I'), value='AC'), Output()), _…

In [43]:
# Turns off warning when using np.where to copy values
pd.options.mode.chained_assignment = None

### 4. Adding Interactivity to Geospatial Visuals

In [44]:
def m_dist(Market, Product, Attribute, Threshold):
    d = df[(df.media_product_name == Product) & (df.sales_market == Market) & (df.DoW == 'total')]
    d['label'] = np.where(d[Attribute] >= Threshold,0, 1)
    
    color = ['red','blue']
    
    map_val = folium.Map(location=[round(d['lat'].mean(), 2),
                                   round(d['lon'].mean(), 2)], 
                     zoom_start=10, 
                     tiles='Stamen Toner')

    for row in d.iterrows():
        row_values = row[1]
        location = [row_values['lat'], row_values['lon']]
        
        html = html = '<h4> <b> Board ID: ' + str(row_values['uid_panel']) + '</h4></b>' + '<br><b>Audience Composition: </b>' + str(row_values['audComp']) + '<br><br><b> Index: </b>' + str(row_values['index'])
        iframe = folium.IFrame(html=html, width=300, height=150)
        popup = folium.Popup(iframe, max_width=2650)
        
        marker = folium.Marker(location=location, popup=popup, icon=folium.Icon(color=color[row_values['label']]))
        marker.add_to(map_val)
        
        
    return map_val
        
# Options for values included here
temp = widgets.interact(m_dist,
    Attribute = widgets.ToggleButtons(
    options=['AC', 'AD', 'I'],
    description='Index:',
    disabled=False,
    button_style=''),
         Market = widgets.Dropdown(
             options = df.sales_market.unique(),
             value = 'Toronto CMA',
             descriptions = 'Market'),
         Product = widgets.Dropdown(
             options = df.media_product_name.unique(),
             value = 'Horizontal',
             descriptions = 'Product'),
        Threshold = widgets.FloatSlider(
            value=0.5,
            min = -1,
            max=1,
            step=0.05,
            disabled = False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'))

interactive(children=(Dropdown(description='Market', index=2, options=('Quebec CMA', 'Ottawa CMA', 'Toronto CM…

### 5. Customization of HTMLs

In [45]:
d = df[(df.media_product_name == 'Horizontal') & (df.sales_market == 'Toronto CMA') & (df.DoW == 'total')]

map_val = folium.Map(location=[round(d['lat'].mean(), 2),
                                   round(d['lon'].mean(), 2)], 
                     zoom_start=10, 
                     tiles='Stamen Toner')

# To Be Fixed

In [46]:
for row in d.iterrows():
        row_values = row[1]
        location = [row_values['lat'], row_values['lon']]
        uid_panel = row_values['uid_panel']
        html=f"""
        <h1> <b>Board ID: {uid_panel}<b></h1> 
        """
        iframe = folium.IFrame(html=html, width=500, height=300)
        popup = folium.Popup(iframe, max_width=2650)
        marker = folium.Marker(location=location, popup=popup)
        marker.add_to(map_val)
        

In [47]:
map_val